In [102]:
from dataset import WelQrateDataset
from mol_utils.scaffold_split import generate_scaffolds
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import numpy as np

In [2]:
name = "AID1798"
dataset_2d = WelQrateDataset(name, root ='../dataset_test', mol_repr ='2dmol', task_type='classification')
smiles_list = dataset_2d.smiles
print(len(smiles_list))

dataset stored in ../dataset_test/AID1798
AID1798 dataset files already exist. Skipping download.
60706


In [93]:
active_smiles = [data.smiles for data in dataset_2d if data.y == 1]
all_smiles = [data.smiles for data in dataset_2d]

In [95]:
active_scaffold_set,_ = generate_scaffolds(active_smiles)
all_scaffold_set,_ = generate_scaffolds(all_smiles)

100%|██████████| 164/164 [00:00<00:00, 4366.60it/s]


Number of scaffold generated: 144
Number of molecules not parsed: 0


100%|██████████| 60706/60706 [00:11<00:00, 5074.71it/s]

Number of scaffold generated: 30079
Number of molecules not parsed: 0


In [98]:
active_scaffold_list = list(active_scaffold_set.keys())
active_scaffold_mols = [Chem.MolFromSmiles(s) for s in active_scaffold_list]

all_scaffold_list = list(all_scaffold_set.keys())
all_scaffold_mols = [Chem.MolFromSmiles(s) for s in all_scaffold_list]

In [99]:
def calculate_ecfp(scaffold_mols):
    ecfp = []
    error = []
    for mol in scaffold_mols:
        if mol is None:
            print('Error: None molecule')
            error.append(mol)
            ecfp.append([None]*1024)
        else:
            list_bits_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useFeatures=True)
            ecfp.append(list_bits_fingerprint)
    return ecfp, error

In [101]:
active_ecfp, _ = calculate_ecfp(active_scaffold_mols)
all_ecfp, _ = calculate_ecfp(all_scaffold_mols)

In [142]:
sim_matrix  = np.zeros((len(active_ecfp), len(all_ecfp)))
sim_matrix.shape

(144, 30079)

In [143]:
for i in range(sim_matrix.shape[0]):
    for j in range(i+1, sim_matrix.shape[1]):
        if active_ecfp[i] is None or all_ecfp[j] is None:
            sim_matrix[i,j] = None
        else:
            sim_matrix[i,j] = DataStructs.TanimotoSimilarity(active_ecfp[i], all_ecfp[j])

In [144]:
sim_matrix

array([[0.        , 0.16666667, 0.14285714, ..., 0.22222222, 0.13043478,
        0.125     ],
       [0.        , 0.        , 0.14545455, ..., 0.16981132, 0.11320755,
        0.10638298],
       [0.        , 0.        , 0.        , ..., 0.16949153, 0.17857143,
        0.15686275],
       ...,
       [0.        , 0.        , 0.        , ..., 0.2       , 0.19117647,
        0.19354839],
       [0.        , 0.        , 0.        , ..., 0.0754717 , 0.10204082,
        0.11904762],
       [0.        , 0.        , 0.        , ..., 0.13559322, 0.14285714,
        0.14      ]])

In [145]:
print(np.max(sim_matrix, axis=1))

[1.         0.71875    0.70731707 0.5        0.70588235 0.625
 0.62068966 0.51428571 0.79411765 0.76086957 0.71794872 0.56410256
 0.59459459 0.75862069 1.         0.62222222 0.80851064 0.78571429
 1.         1.         0.74074074 0.78947368 1.         0.97058824
 1.         0.8        0.64516129 0.89285714 0.57142857 0.78378378
 0.97826087 0.71111111 0.75757576 0.84210526 0.70909091 0.725
 0.78       0.66666667 0.64516129 0.55172414 0.90909091 0.80392157
 0.77777778 0.86956522 0.9047619  0.69565217 0.58823529 1.
 1.         0.92307692 1.         0.58823529 0.69230769 1.
 1.         0.86666667 0.85714286 0.75       0.81818182 0.65714286
 0.72727273 1.         0.76923077 0.93939394 0.71794872 0.97368421
 0.46511628 0.86046512 0.86206897 0.89285714 1.         0.94444444
 0.73170732 0.87179487 0.65517241 0.76923077 0.97058824 0.8125
 1.         1.         0.65714286 0.55319149 0.7826087  0.8
 0.58333333 0.90625    0.64285714 0.72       0.51612903 0.81578947
 0.65789474 0.70967742 0.78     